<a href="https://colab.research.google.com/github/YazzRz/TFM_Ana_Rozo_Georeferenciacion_Violencia_Machista/blob/main/Revisi%C3%B3n_georreferenciaci%C3%B3n_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade google-cloud-aiplatform google-cloud-bigquery panda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 59.3 MB/s eta 0:00:00
  Created wheel for panda: filename=panda-0.3.1-py3-none-any.whl size=7239 sha256=356ff76e75ce02dd65bd936fb31c2ed0eba3bba3787b150d899ffa255fad8c0e
  Stored in directory: /root/.cache/pip/wheels/98/41/5b/6ca54e0b6a35e1b7248c12f56fcb753dfb7717fefaa0fb45f5
Successfully built panda
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.118.0
    Uninstalling google-cloud-aiplatform-1.118.0:
      Successfully uninstalled google-cloud-aiplatform-1.118.0


In [1]:
from vertexai import init
from vertexai.generative_models import GenerativeModel
from google.cloud import bigquery
import pandas as pd


In [2]:
# Inicializar Vertex AI y BigQuery
init(project="apt-arcana-472403-u5", location="us-central1")
bq = bigquery.Client(project="apt-arcana-472403-u5")

# Cargar el modelo de Gemini
model = GenerativeModel("gemini-1.5-pro")


/usr/local/lib/python3.12/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [8]:
!gcloud auth application-default login
!gcloud auth application-default set-quota-project apt-arcana-472403-u5

from vertexai import init
init(project="apt-arcana-472403-u5", location="us-central1")


Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=96ztHlaq0UoqTbcbNUSpKPqRkRLJeo&prompt=consent&token_usage=remote&access_type=offline&code_challenge=vqyv8bZwLpf205surZMA22O8cG7Ediy3YaajUkk5iTA&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AVGzR1DjVq9-MaERoG7eO0p7oxhUPuzqxNMsIIASnKT0X5i4MATFw5ZIlv-AyZymY1_9aA

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [12]:
from google.auth import default
from vertexai import init
from vertexai.generative_models import GenerativeModel
from google.cloud import bigquery

# --- Configuración ---
PROJECT_ID = "apt-arcana-472403-u5"
REGION = "us-central1"

# --- Cargar credenciales OAuth ---
creds, _ = default()

# --- Inicializar Vertex y BigQuery con credenciales explícitas ---
init(project=PROJECT_ID, location=REGION, credentials=creds)
bq = bigquery.Client(project=PROJECT_ID, credentials=creds)

# --- Cargar modelo Gemini ---
model = GenerativeModel("gemini-2.5-pro")
print("✅ Modelo gemini-2.5-pro cargado correctamente.")

# --- Prueba rápida ---
texto_prueba = "El presidente viajó a Buenos Aires para reunirse con líderes regionales."
prompt = f"Identifica el país mencionado o implícito en este texto. Devuelve solo el nombre del país:\n{texto_prueba}"

respuesta = model.generate_content(prompt)
print("🌎 Resultado de prueba:", respuesta.text)


/usr/local/lib/python3.12/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


✅ Modelo gemini-2.5-pro cargado correctamente.
🌎 Resultado de prueba: Argentina


In [13]:
query = """
SELECT
P.ID_noticia,
contenido,
FROM (
SELECT
ID_noticia,
contenido,
JSON_VALUE(ubicacion, "$.País") AS pais_detectado,
JSON_VALUE(ubicacion, "$.Departamento") AS departamento_detectado,
JSON_VALUE(ubicacion, "$.Municipio") AS municipio_detectado,
REGEXP_EXTRACT_ALL(tokens_detectados, r'"([^"]+)":') AS token_keys,
REGEXP_EXTRACT_ALL(tokens_detectados, r':\s*([0-9]+)') AS token_vals
FROM `apt-arcana-472403-u5.TFM.consolidado_periodicos`
) as P,
UNNEST(token_keys) AS key WITH OFFSET pos
JOIN UNNEST(token_vals) AS val WITH OFFSET pos2
ON pos = pos2
where val != '0' and P.pais_detectado =""
GROUP BY 1,2
"""
df = bq.query(query).to_dataframe()
print(df)


<>:13: SyntaxWarning: invalid escape sequence '\s'
<>:13: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-3696624559.py:13: SyntaxWarning: invalid escape sequence '\s'
  REGEXP_EXTRACT_ALL(tokens_detectados, r':\s*([0-9]+)') AS token_vals


                ID_noticia                                          contenido
0      CO_20200903220241_0   A mediados de agosto, el periódicoVanguardiac...
1      CO_20160216160728_0   Abuso de menores Meta, Quindío, Amazonas y Ri...
2      CO_20210612092221_1   Acá encontrarás tus noticias de Revisa tu ban...
3      CO_20211122232439_2   Acá encontrarás tus noticias de Revisa tu ban...
4      CO_20220313232337_1   Acá encontrarás tus noticias de Revisa tu ban...
...                    ...                                                ...
1908  ESP_20190623204344_1  Las víctimas de violencia de género aumentaron...
1909  ESP_20190613144832_1  Las víctimas de violencia de género aumentaron...
1910  ESP_20190614075724_3  Las víctimas de violencia de género aumentaron...
1911  ESP_20190628081418_5  Las víctimas de violencia de género aumentaron...
1912  ESP_20190706110602_1  Las víctimas de violencia de género aumentaron...

[1913 rows x 2 columns]


In [14]:
def detectar_ubicacion_desagregada_gemini(texto):
    if not texto or len(texto.strip()) == 0:
        return {
            "pais": "DESCONOCIDO",
            "departamento": "DESCONOCIDO",
            "municipio": "DESCONOCIDO"
        }

    prompt = f"""
    Analiza el siguiente texto y determina la ubicación geográfica principal mencionada o implícita.

    Devuelve solo los **nombres** en este formato exacto (sin explicaciones, sin comillas y separados por '|'):
    País | Departamento / Comunidad Autónoma | Ciudad / Municipio

    Si alguno no se puede identificar, escribe DESCONOCIDO.

    Ejemplo de salida esperada:
    Colombia | Antioquia | Medellín
    España | Comunidad de Madrid | Madrid
    México | Chihuahua | Ciudad Juárez
    Chile | DESCONOCIDO | Santiago

    Texto:
    {texto}
    """

    try:
        respuesta = model.generate_content(prompt)
        partes = [p.strip() for p in respuesta.text.split("|")]
        while len(partes) < 3:
            partes.append("DESCONOCIDO")

        return {
            "pais": partes[0].replace('"', '').strip(),
            "departamento": partes[1].replace('"', '').strip(),
            "municipio": partes[2].replace('"', '').strip()
        }
    except Exception as e:
        return {
            "pais": f"Error: {str(e)}",
            "departamento": "DESCONOCIDO",
            "municipio": "DESCONOCIDO"
        }


In [15]:
import pandas as pd
import time
from google.cloud import bigquery

# --- Configuración ---
PROJECT_ID = "apt-arcana-472403-u5"
DATASET_ID = "TFM"
TABLE_ID = "ubicaciones_noticias_gemini"
TABLE_REF = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

bq_client = bigquery.Client(project=PROJECT_ID)

# --- Esquema de la tabla ---
schema = [
    bigquery.SchemaField("ID_noticia", "STRING"),
    bigquery.SchemaField("contenido", "STRING"),
    bigquery.SchemaField("pais", "STRING"),
    bigquery.SchemaField("departamento", "STRING"),
    bigquery.SchemaField("municipio", "STRING"),
]

# --- Crear tabla si no existe ---
try:
    bq_client.get_table(TABLE_REF)
    print(f"✅ La tabla ya existe: {TABLE_REF}")
except Exception:
    table = bigquery.Table(TABLE_REF, schema=schema)
    table = bq_client.create_table(table)
    print(f"🆕 Tabla creada: {TABLE_REF}")

# --- Función para insertar registros ---
def insertar_en_bigquery(registro):
    errors = bq_client.insert_rows_json(TABLE_REF, [registro])
    if errors:
        print(f"⚠️ Error al insertar en BigQuery: {errors}")

# --- Procesamiento con progreso ---
total = len(df)
inicio = time.time()

for i, row in df.iterrows():
    try:
        # 1️⃣ Llamar a Gemini
        ubicacion = detectar_ubicacion_desagregada_gemini(row["contenido"])

        # 2️⃣ Crear registro
        registro = {
            "ID_noticia": str(row["ID_noticia"]),
            "contenido": row["contenido"],
            "pais": ubicacion.get("pais"),
            "departamento": ubicacion.get("departamento"),
            "municipio": ubicacion.get("municipio"),
        }

        # 3️⃣ Insertar en BigQuery
        insertar_en_bigquery(registro)

        # 4️⃣ Mostrar progreso
        if (i + 1) % 10 == 0 or i == total - 1:
            elapsed = time.time() - inicio
            print(f"✅ {i+1}/{total} ({(i+1)/total:.1%}) completado — {elapsed:.1f}s")

    except Exception as e:
        print(f"❌ Error en línea {i+1}: {e}")
        continue


🆕 Tabla creada: apt-arcana-472403-u5.TFM.ubicaciones_noticias_gemini
✅ 10/1913 (0.5%) completado — 177.7s
✅ 20/1913 (1.0%) completado — 342.5s
✅ 30/1913 (1.6%) completado — 487.9s
✅ 40/1913 (2.1%) completado — 746.7s
✅ 50/1913 (2.6%) completado — 925.1s
✅ 60/1913 (3.1%) completado — 1043.2s
✅ 70/1913 (3.7%) completado — 1174.3s
✅ 80/1913 (4.2%) completado — 1317.0s
✅ 90/1913 (4.7%) completado — 1469.7s
✅ 100/1913 (5.2%) completado — 1668.6s
✅ 110/1913 (5.8%) completado — 1863.3s
✅ 120/1913 (6.3%) completado — 2035.8s
✅ 130/1913 (6.8%) completado — 2118.2s
✅ 140/1913 (7.3%) completado — 2219.8s
✅ 150/1913 (7.8%) completado — 2332.7s
✅ 160/1913 (8.4%) completado — 2557.7s
✅ 170/1913 (8.9%) completado — 2727.7s
✅ 180/1913 (9.4%) completado — 2885.8s
✅ 190/1913 (9.9%) completado — 3023.0s
✅ 200/1913 (10.5%) completado — 3116.9s
✅ 210/1913 (11.0%) completado — 3266.1s
✅ 220/1913 (11.5%) completado — 3482.7s
✅ 230/1913 (12.0%) completado — 3718.6s
✅ 240/1913 (12.5%) completado — 3882.8s
✅ 25

In [ ]:
# ==========================================
# 🔹 GUARDAR RESULTADOS DE GEMINI EN BIGQUERY
# ==========================================

from google.cloud import bigquery
import pandas as pd

# Parámetros
PROJECT_ID = "apt-arcana-472403-u5"
DATASET = "TFM"
TABLE_NAME = "lugares_detectados_gemini"
DESTINATION = f"{DATASET}.{TABLE_NAME}"

# Guardar DataFrame en BigQuery
df.to_gbq(destination_table=DESTINATION, project_id=PROJECT_ID, if_exists="replace")

# Confirmar carga
bq = bigquery.Client(project=PROJECT_ID)
query = f"SELECT COUNT(*) AS total_filas FROM `{PROJECT_ID}.{DESTINATION}`"
total = bq.query(query).to_dataframe().iloc[0, 0]
print(f"✅ Tabla '{DESTINATION}' creada correctamente con {total} filas.")


/tmp/ipython-input-3689747778.py:15: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(destination_table=DESTINATION, project_id=PROJECT_ID, if_exists="replace")
100%|██████████| 1/1 [00:00<00:00, 9300.01it/s]


✅ Tabla 'TFM.lugares_detectados_gemini' creada correctamente con 10 filas.
